<a href="https://colab.research.google.com/github/Paushi2003/Paddy_Pest_Disease_Classification/blob/main/plant_disease_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#connect google drive to load training images
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import keras
import tensorflow
from keras.preprocessing.image  import ImageDataGenerator
from keras.layers import Flatten, Dense
from keras import layers,models
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
'''
I have the paddy pest disease classification dataset in my drive.
ImageDataGenerator can apply various transformations, such as rotation, flipping, scaling, shearing,
and more, to generate new variations of the input images in real-time.
This helps the model to generalize better.
'''
train_datagen = ImageDataGenerator(zoom_range=0.5,shear_range=0.3,rescale=1/255,horizontal_flip=True)
train = train_datagen.flow_from_directory(directory='drive/My Drive/paddy-disease-classification/train_images',target_size=(256,256),batch_size=32)


In [ ]:
'''
Showcasing three images from the dataset
'''
t_img,label = train.next()
def plotImage(imgarr,label):
  for im,l in zip(imgarr,label):
    plt.figure(figsize=(5,5))
    plt.imshow(im)
    plt.show()

plotImage(t_img[:3],label[:3])

In [ ]:
'''
This is a CNN model that contains 12 layers of which 6 are conv2D layers and the other are MaxPooling2D layers.
They are stacked one above the other alternatively. I have set the input shape to 256x256.
I resized the images previously to 256x256 so I'm using the input shape as 256x256.
Instead of relu we can also use tanh, sigmoid, leaky relu and many more.
Since ours is a multi-class classification model that contains 10 classes in total, I am usinf softmax at the output layer.
At the end of 100th epoch I got nearly 74% accuracy.
I have used adam optimizer as that is basically a common optimizer that we use.
Other than adam one can use SGD, AdaGrad, AdaDelta, RMSProp and so on.
Similarly we have used categorical crossentropy as the loss function.
Since this is a multiclass classification, we used categorical crossentropy.
'''
model = models.Sequential([
    layers.Conv2D(128, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128 (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
])
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train, epochs=100,steps_per_epoch=100)
model.save('model_extended.h5')
model.save('model.keras')

In [ ]:
'''
In the above cell, I saved the model as model.h5
In this cell, I'm loading the model and using some preprocessing techniques to preprocess
the input image that we are going to pass as input to our model to predict its class.
Since in the model architecture, I provided input dimension to be 256x256, I am reading the image file,
converting it into array and then resizing it to 256x256.
I am also normalizing the images before passing it into the model.
The model returns the output between 0 to 9.
I'm using that as index value to retrieve the class of image.
'''
img = load_img('/content/100042.jpg', target_size=(256, 256))
img_array = img_to_array(img)
img_array = img_array.reshape((1, 256, 256, 3))
img_array = img_array / 255.0
predictions = model.predict(img_array)
predicted_class_index = predictions.argmax()
class_labels = ['bacterial_leaf_blight', 'bacterial_leaf_streak', 'bacterial_panicle_blight','blast','brown_spot','dead_heart','downy_mildew','hispa','normal','tungro' ]  # Replace with your actual class labels
predicted_class_label = class_labels[predicted_class_index]
print(predicted_class_label)